# LGB 



 ## Import



In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from tqdm import tnrange, tqdm_notebook

import lightgbm as lgb
from lightgbm import LGBMRegressor

import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
rc('font', family='AppleGothic')
import warnings
warnings.filterwarnings('ignore')


In [3]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


In [4]:
del_col = ['ID','판매구역']
enc_col = [ '브랜드', '차량모델명', '판매도시']

In [5]:
train = train.drop(columns = del_col)
test = test.drop(columns = del_col)

## Encoder

In [6]:
for i in enc_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    train[i]=le.transform(train[i])
    test[i]=le.transform(test[i])

## X_train, y_train

In [7]:
y_train = train['가격']
X_train = train.drop(columns=['가격'])

In [8]:
X_K=X_train.copy()
y_K=y_train.copy()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,
                                                    test_size=0.2, random_state=42)

In [8]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(46336, 12) (46336,)
(11584, 12) (11584,)


## Optuna

In [44]:
def objective3(trial):

    param = {
        'random_state': trial.suggest_int('random_state',10,50),
        'n_estimators': trial.suggest_int('n_estimators',500,2200),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
        'max_depth': trial.suggest_categorical('max_depth', [5,10,20,40,50,100, -1]),
        # 'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = LGBMRegressor(**param, n_jobs=-1)
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    rmse = mean_absolute_error(pred,y_val)

    return rmse

In [45]:
study3 = optuna.create_study(direction='minimize', sampler=TPESampler())

# n_trials 지정해주지 않으면, 무한 반복
study3.optimize( objective3, n_trials = 100)

print('Best trial : score {}, \nparams {}'.format(study3.best_trial.value, study3.best_trial.params))

[I 2023-06-24 01:32:37,657] A new study created in memory with name: no-name-527a3430-56c1-4a6d-bb13-727d669c9c60
[I 2023-06-24 01:32:39,421] Trial 0 finished with value: 6.033477568734807 and parameters: {'random_state': 15, 'n_estimators': 772, 'reg_alpha': 4.955727413167556, 'reg_lambda': 0.3124775832776758, 'colsample_bytree': 0.831371354159583, 'subsample': 0.6097657705259278, 'learning_rate': 0.09055760327099079, 'max_depth': 100, 'min_child_samples': 252, 'min_data_per_groups': 30}. Best is trial 0 with value: 6.033477568734807.
[I 2023-06-24 01:32:44,807] Trial 1 finished with value: 20.697296858973996 and parameters: {'random_state': 40, 'n_estimators': 2052, 'reg_alpha': 0.0016421763325236945, 'reg_lambda': 0.10844351108332949, 'colsample_bytree': 0.45949085939576034, 'subsample': 0.47305917654327595, 'learning_rate': 0.0002508411053648797, 'max_depth': -1, 'min_child_samples': 12, 'min_data_per_groups': 92}. Best is trial 0 with value: 6.033477568734807.
[I 2023-06-24 01:32:

Best trial : score 5.783093799907807, 
params {'random_state': 13, 'n_estimators': 2108, 'reg_alpha': 0.10870015225264901, 'reg_lambda': 0.003993098123868187, 'colsample_bytree': 0.4956652917027736, 'subsample': 0.7084359141500247, 'learning_rate': 0.05205024733601091, 'max_depth': 20, 'min_child_samples': 6, 'min_data_per_groups': 40}


In [46]:
study3.best_trial.params

{'random_state': 13,
 'n_estimators': 2108,
 'reg_alpha': 0.10870015225264901,
 'reg_lambda': 0.003993098123868187,
 'colsample_bytree': 0.4956652917027736,
 'subsample': 0.7084359141500247,
 'learning_rate': 0.05205024733601091,
 'max_depth': 20,
 'min_child_samples': 6,
 'min_data_per_groups': 40}

In [8]:
model_lgb3 = LGBMRegressor(random_state=13,
                           n_estimators= 2108,
                           reg_alpha = 0.10870015225264901,
                           reg_lambda = 0.003993098123868187,
                           colsample_bytree = 0.4956652917027736,
                           subsample = 0.7084359141500247,
                           learning_rate = 0.05205024733601091,
                           max_depth = 20,
                           min_child_samples = 6,
                           min_data_per_groups = 40)

In [9]:
model_lgb3.fit(X_train,y_train)

[LightGBM] [Warning] Unknown parameter: min_data_per_groups


LGBMRegressor(colsample_bytree=0.4956652917027736,
              learning_rate=0.05205024733601091, max_depth=20,
              min_child_samples=6, min_data_per_groups=40, n_estimators=2108,
              random_state=13, reg_alpha=0.10870015225264901,
              reg_lambda=0.003993098123868187, subsample=0.7084359141500247)

In [14]:
pred3=model_lgb3.predict(test)

# K-fold & Optuna

In [19]:
def optimization_k2(trial):
    score = []
    kf = KFold(n_splits = 5 , random_state = 42 , shuffle = True)
    for train_fold, test_fold in tqdm_notebook(kf.split(X_K, y_K), desc = 'k_fold'):
        X_train, X_test, y_train, y_test = X_K.iloc[train_fold], X_K.iloc[test_fold], y_K[train_fold], y_K[test_fold] 
        

        param = {
            'random_state': trial.suggest_int('random_state',10,50),
            'n_estimators': trial.suggest_int('n_estimators',1000,2500),
            'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1.0),
            'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1.0),
            'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 1.0),
            'subsample': trial.suggest_loguniform('subsample', 0.4, 1.0),
            'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 0.5),
            'max_depth': trial.suggest_categorical('max_depth', [5,10,20,40,50,100, -1]),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
            'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
        }
        model = LGBMRegressor(**param, n_jobs=-1)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        score_cv = mean_absolute_error(pred,y_test)
        
        score.append(score_cv)

    return np.mean(score)
    return np.mean(score)

In [20]:
study_k2 = optuna.create_study(direction='minimize', sampler=TPESampler())

# n_trials 지정해주지 않으면, 무한 반복
study_k2.optimize( objective_k, n_trials = 100)

print('Best trial : score {}, \nparams {}'.format(study_k2.best_trial.value, study_k2.best_trial.params))

[I 2023-06-25 16:39:43,851] A new study created in memory with name: no-name-c6f7ebd3-299e-475b-96c7-3f0995a72d61


k_fold: 0it [00:00, ?it/s]

[I 2023-06-25 16:39:59,189] Trial 0 finished with value: 14.054400444617794 and parameters: {'random_state': 15, 'n_estimators': 1637, 'reg_alpha': 0.008628994264796663, 'reg_lambda': 0.09807106140228726, 'colsample_bytree': 0.2182905017244789, 'subsample': 0.8442420434123656, 'learning_rate': 0.0016675015740205523, 'max_depth': 40, 'min_child_samples': 256, 'min_data_per_groups': 89}. Best is trial 0 with value: 14.054400444617794.


k_fold: 0it [00:00, ?it/s]

[I 2023-06-25 16:40:15,027] Trial 1 finished with value: 6.561444923248357 and parameters: {'random_state': 27, 'n_estimators': 1480, 'reg_alpha': 0.5061919460221151, 'reg_lambda': 1.5276730654285318, 'colsample_bytree': 0.2922144838127072, 'subsample': 0.6818010203264399, 'learning_rate': 0.012676858903132523, 'max_depth': 40, 'min_child_samples': 129, 'min_data_per_groups': 43}. Best is trial 1 with value: 6.561444923248357.


...

k_fold: 0it [00:00, ?it/s]

[I 2023-06-25 16:59:08,498] Trial 98 finished with value: 5.878958306491298 and parameters: {'random_state': 37, 'n_estimators': 2107, 'reg_alpha': 3.6416802705835907, 'reg_lambda': 4.0270947034818905, 'colsample_bytree': 0.4580417301265054, 'subsample': 0.4161465990135346, 'learning_rate': 0.13891444132348352, 'max_depth': -1, 'min_child_samples': 15, 'min_data_per_groups': 20}. Best is trial 92 with value: 5.7940823146179685.


k_fold: 0it [00:00, ?it/s]

[I 2023-06-25 16:59:25,100] Trial 99 finished with value: 5.967374072706693 and parameters: {'random_state': 30, 'n_estimators': 1896, 'reg_alpha': 5.625085316679212, 'reg_lambda': 2.987130863671828, 'colsample_bytree': 0.41696277930735504, 'subsample': 0.4587889505683855, 'learning_rate': 0.052061659760838055, 'max_depth': 50, 'min_child_samples': 219, 'min_data_per_groups': 12}. Best is trial 92 with value: 5.7940823146179685.


Best trial : score 5.7940823146179685, 
params {'random_state': 33, 'n_estimators': 2141, 'reg_alpha': 7.4359260960440565, 'reg_lambda': 3.750025461043171, 'colsample_bytree': 0.46146033706677003, 'subsample': 0.4693307894458788, 'learning_rate': 0.09777125124885924, 'max_depth': -1, 'min_child_samples': 7, 'min_data_per_groups': 32}


In [21]:
study_k2.best_trial.params

{'random_state': 33,
 'n_estimators': 2141,
 'reg_alpha': 7.4359260960440565,
 'reg_lambda': 3.750025461043171,
 'colsample_bytree': 0.46146033706677003,
 'subsample': 0.4693307894458788,
 'learning_rate': 0.09777125124885924,
 'max_depth': -1,
 'min_child_samples': 7,
 'min_data_per_groups': 32}

In [ ]:
model_lgb_k2 = LGBMRegressor(random_state=33,
                           n_estimators= 2141,
                           reg_alpha = 7.4359260960440565,
                           reg_lambda = 3.750025461043171,
                           colsample_bytree =0.46146033706677003,
                           subsample =0.4693307894458788,
                           learning_rate = 0.09777125124885924,
                           max_depth = -1,
                           min_child_samples = 7,
                           min_data_per_groups = 32)

In [ ]:
model_lgb_k2.fit(X_train,y_train)

In [ ]:
pred_k=model_lgb_k2.predict(test)

## Ensemble
- xgb,lgb 

In [24]:
pred_xgb = pd.read_csv('submit_xgb2.csv')
pred_xgb1=pred_xgb['가격']
pred_xgb_k= pd.read_csv('submit_xgb_K.csv')
pred_xgb2=pred_xgb_k['가격']


In [ ]:
pred_a=(pred3+pred_xgb1+pred_xgb2+pred_k)/4

In [34]:
sample_submission = pd.read_csv('DATA/sample_submission.csv')
sample_submission['가격'] = pred_a
sample_submission.to_csv("submit_xgb_lgb4.csv", index = False)